<a href="https://colab.research.google.com/github/CalCraven/plotting_in_python/blob/main/plot-Membrane-Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_miniforge()

In [ ]:
import condacolab
condacolab.check()

!conda install mamba -y
!mamba install anaconda-client -n base -y

!git clone https://github.com/CalCraven/plotting_in_python.git
%cd plotting_in_python

%cd ..
!pip install --upgrade ipykernel -y

!mamba install -c conda-forge pandas plotly matplotlib numpy openpyxl jupyterlab kaleido -y

## Computer DataTypes in Python

In [ ]:
# numeric types:
a = 5
print("Type of a: ", type(a))

b = 5.0
print("\nType of b: ", type(b))

c = 2 + 4j
print("\nType of c: ", type(c))


In [ ]:
# string types:
String1 = 'Welcome to the Geeks World'
print("String with the use of Single Quotes: ")
print(String1)
String1 = "I'm a Geek"
print("\nString with the use of Double Quotes: ")
print(String1)
print(type(String1))
String1 = '''I'm a Geek and I live in a world of "Geeks"'''
print("\nString with the use of Triple Quotes: ")
print(String1)
print(type(String1))

String1 = '''Geeks
            For
            Life'''
print("\nCreating a multiline String: ")
print(String1)

In [ ]:
# container types:
List = []
print("Initial blank List: ")
print(List)
List = ['GeeksForGeeks']
print("\nList with the use of String: ")
print(List)
List = ["Geeks", "For", "Geeks"]
print("\nList containing multiple values: ")
print(List[0])
print(List[2])
List = [['Geeks', 'For'], ['Geeks']]
print("\nMulti-Dimensional List: ")
print(List)

In [ ]:
# accessing containers
List = ["Geeks", "For", "Geeks"]
print("Accessing element from the list")
print(List[0])
print(List[2])
print("Accessing element using negative indexing")
print(List[-1])
print(List[-3])

In [ ]:
# Numpy (importing pacakges)
import numpy as np

List = ["Geeks", "For", "Geeks"]
Array = np.array(List)

print(Array, type(Array))
print(Array[2])

# math operations with numpy
Array = np.array([1,2,3,5])
print(Array*2)
print(Array-1)



In [ ]:
# Working with functions
Array = np.array([1,2,3,5])

# manually define functions
def get_mean(values):
    current_sum = 0
    for i in values:
        current_sum = current_sum + i
    return i / len(Array)
print("Manual Mean: ", get_mean(Array))

# class functions
print("Class Mean: ", np.mean(Array), "Class STD: ", np.std(Array).round(2))

## Working with datasets in Python

In [ ]:
# colleted_data
import numpy as np
import pandas as pd

np.seed = 12345 # set controllable seed

feedA = np.array([
    0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2,
    0.3,  0.3,  0.3, 0.4, 0.4, 0.4,
    0.5, 0.5, 0.5
]) # weight percentage
feedB = 1-feedA # weight percentage
permA = np.array([
    0.8, 0.81, 0.82, 0.84, 0.85, 0.84,
    0.84, 0.88, 0.87, 0.89, 0.9,
    0.91, 0.891, 0.92, 0.91, 0.92,
]) # weight percentage
permB = 1-permA # weight percentage
feedFlow = np.random.sample(len(feedA)) * 2000
permFlow = np.random.sample(len(feedA)) * 500

rawDF = pd.DataFrame(
    data=np.array([feedA, feedB, permA, permB, feedFlow, permFlow]).T,
    columns=["feedA", "feedB", "permA", "permB", "feedFlow", "permFlow"]
)
rawDF

In [ ]:
# calculate mean and std (built-in functions)
# result = rawDF.groupby(['feedA'], as_index=False).agg(
#     {'permA':['mean','std']}
# )
result = rawDF.groupby(['feedA'])['permA'].mean()
result

In [ ]:
# use a loop to calculate all means
meansDF = pd.DataFrame()
for group in ["feedA", "feedB", "permA", "permB", "feedFlow", "permFlow"]:
    result = rawDF.groupby(['feedA'])[group].mean()
    meansDF[group] = result
meansDF

In [ ]:
# use a loop to calculate all standard deviations
for group in ["feedA", "feedB", "permA", "permB", "feedFlow", "permFlow"]:
    result = rawDF.groupby(['feedA'])[group].std()
    meansDF[group+"_std"] = result
meansDF

In [ ]:
# calculate parameters (make-your own functions)
# selectivity
def selectivity(permA):
    """Calculated membrane selectivity.
    https://personalpages.manchester.ac.uk/staff/Peter.Budd/SELECTIVITY%20&%20PERMEABILITY.htm

    selectivity_AB = permeabilityA/permeabilityB

    permeabilityA = fluxA * membrane thickness / dPressure

    selectivity = fluxA / fluxB
    selectivity = permA * permFlow / (1-permA) / permFlow
    selectivity = permA / (1-permA)
    """
    return permA / (1-permA)
meansDF["selectivity"] = meansDF.apply(lambda x: selectivity(x['permA']), axis=1)
meansDF

In [ ]:
# propogate errors
def selectivity_deviation(permAstd, permAmean):
    """Calculated membrane selectivity standard deviation.

    Sources
    https://chem.libretexts.org/Bookshelves/Analytical_Chemistry/Supplemental_Modules_(Analytical_Chemistry)/Quantifying_Nature/Significant_Digits/Propagation_of_Error
    https://personalpages.manchester.ac.uk/staff/Peter.Budd/SELECTIVITY%20&%20PERMEABILITY.htm
    variables = permA, permFlow
    selectivity = "permA*permFlow/(1-permA)/permFlow"
    selectivity = permA/(1-permA)
    VAR(selectivity)/EXP(selectivity)**2 = (VAR(permA)/EXP(permA)**2) *2
    VAR(selectivity) = 2 * (VAR(permA)/EXP(permA)**2) * EXP(selectivity)**2
    VAR(X) = STD(X)**2
    STD(selectivity) = sqrt(2) * STD(permA) * EXP(selectivity) / EXP(permA)
    """

    return np.sqrt(2) * permAstd * selectivity(permAmean) / permAmean

meansDF["selectivity_std"] = meansDF.apply(lambda x: selectivity_deviation(x['permA_std'], x['permA']), axis=1)
meansDF

## Plot Real Data
Data collected by Matthew Vasuta for fluorinated polymer systems.

In [ ]:
# Load data
import pandas as pd

df = pd.read_excel('plotting_in_python/data/7-2-24 Membrane Graphs.xlsx', header=1)
df

In [ ]:
# single scatter
import plotly.io as pio
import plotly.express as px

pio.templates.default = "simple_white"
fig = px.scatter(
    df, x="% Fluor ination", y="α", color="β",
    error_y="Unnamed: 11",
)
fig.update_traces(
    marker=dict(size=16,
    line=dict(width=2,
    color='DarkSlateGrey')),
    selector=dict(mode='markers')
)
fig.show(renderer="colab")
fig.write_image("figures/Selectivity-vs-permeability.pdf")


In [ ]:
# plotting multi-variate bits and bobs
# https://www.kaggle.com/code/residentmario/multivariate-plotting
from pandas.plotting import parallel_coordinates
import matplotlib.pyplot as plt
from matplotlib import colormaps as cm

plot_xdata = [
    '% Fluor ination', 'Total Flux (g/(m^2×h))',
    'Water Permeance (GPU)', 'α', 'β', 'Percent EtOH'
]

normDF = df[plot_xdata[1:]].apply(lambda x: ((x.max() - x)/(x.max() - x.min())))
legend_label = '% Fluor ination'
normDF[legend_label] = df[legend_label]
cmap = cm["plasma"]
ax = parallel_coordinates(
    normDF[plot_xdata], legend_label, colormap=cmap
)
fig = ax.get_figure()
ax.get_legend().remove()

fig.legend(title=legend_label, loc="right", bbox_to_anchor=[1.1,0.7])
plt.xticks(rotation=30, ha="right")
fig.show()